In [2]:
import requests
import string
from collections import Counter

class WordleSolver:
    def __init__(self, base_url="https://wordle.votee.dev:8000"):
        self.base_url = base_url
        self.word_length = 0
        self.seed = None
        self.alphabet = string.ascii_lowercase
        self.word_list = self._load_word_dictionary()
        self.found_letters = []
        self.absent_letters = set()
        self.present_letters = {}

    def _load_word_dictionary(self):
        """Return a list of common words for solving."""
        return [
            'about', 'alert', 'apple', 'beach', 'chair',
            'dance', 'eagle', 'flame', 'grace', 'happy',
            'image', 'juice', 'karma', 'light', 'magic',
            'noble', 'ocean', 'peace', 'quick', 'river',
            'smile', 'tiger', 'unity', 'voice', 'water'
        ]

    def _make_request(self, endpoint, guess):
        """Send a guess request to the API."""
        params = {"guess": guess, "size": self.word_length}
        if self.seed:
            params["seed"] = self.seed
        try:
            response = requests.get(f"{self.base_url}{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error while making request: {e}")
            return []

    def set_parameters(self, length, seed=None):
        """Initialize parameters for the puzzle."""
        self.word_length = length
        self.seed = seed
        self.found_letters = [None] * self.word_length
        self.present_letters = {i: set() for i in range(self.word_length)}
        self.absent_letters = set()

    def calculate_letter_frequency(self):
        """Calculate letter frequency in the word list."""
        counter = Counter()
        for word in self.word_list:
            counter.update(set(word))  # Count each letter only once per word
        return dict(counter)

    def find_letters(self, endpoint):
        """Find existing and correctly placed letters by prioritizing frequent letters."""
        letter_frequencies = self.calculate_letter_frequency()
        sorted_letters = sorted(
            self.alphabet, key=lambda x: letter_frequencies.get(x, 0), reverse=True
        )

        for letter in sorted_letters:
            if letter in self.absent_letters:
                continue
            guess = letter * self.word_length
            print(f"Checking: {guess}")
            results = self._make_request(endpoint, guess)
            if not results:
                continue

            for i, result in enumerate(results):
                if result["result"] == "correct":
                    self.found_letters[i] = letter
                elif result["result"] == "present":
                    self.present_letters[i].add(letter)
                elif result["result"] == "absent":
                    self.absent_letters.add(letter)

    def solve(self, endpoint):
        """Assemble the word based on collected information."""
        guess = [letter if letter else "a" for letter in self.found_letters]

        for i in range(self.word_length):
            if self.found_letters[i] is not None:
                continue
            for letter in self.alphabet:
                if letter in self.absent_letters:
                    continue
                test_guess = guess[:]
                test_guess[i] = letter
                test_word = ''.join(test_guess)
                print(f"Testing word: {test_word}")
                results = self._make_request(endpoint, test_word)
                if not results:
                    continue

                for j, result in enumerate(results):
                    if result["result"] == "correct":
                        self.found_letters[j] = test_guess[j]
                    elif result["result"] == "absent":
                        self.absent_letters.add(test_guess[j])
                break

        final_word = ''.join(self.found_letters)
        print(f"Found word: {final_word}")
        return final_word

def main():
    solver = WordleSolver()
    puzzle_type = input("Choose puzzle type (daily/random): ").strip().lower()
    word_length = int(input("Enter the word length: "))
    seed = None
    if puzzle_type == "random":
        seed = int(input("Enter seed for Random Puzzle: "))

    solver.set_parameters(word_length, seed)
    endpoint = "/daily" if puzzle_type == "daily" else "/random"

    print("\nFinding letters...")
    solver.find_letters(endpoint)

    print("\nSolving the puzzle...")
    solution = solver.solve(endpoint)
    print(f"The word is: {solution}")

if __name__ == "__main__":
    main()


Choose puzzle type (daily/random): random
Enter the word length: 5
Enter seed for Random Puzzle: 44

Finding letters...
Checking: eeeee
Checking: aaaaa
Checking: iiiii
Checking: ccccc
Checking: lllll
Checking: rrrrr
Checking: ggggg
Checking: ttttt
Checking: mmmmm
Checking: hhhhh
Checking: nnnnn
Checking: ooooo
Checking: uuuuu
Checking: bbbbb
Checking: ppppp
Checking: kkkkk
Checking: vvvvv
Checking: yyyyy
Checking: ddddd
Checking: fffff
Checking: jjjjj
Checking: qqqqq
Checking: sssss
Checking: wwwww
Checking: xxxxx
Checking: zzzzz

Solving the puzzle...
Found word: metro
The word is: metro
